In [1]:
SAMPLE_TEXT_1 = """
start-A
start-b
A-c
A-b
b-d
A-end
b-end
"""

SAMPLE_TEXT_2 = """
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
"""

SAMPLE_TEXT_3 = """
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
"""

In [2]:
def tokenize_line(line):
    return line.split('-')


def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]


def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [3]:
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class Node:
    name: str
    is_big: bool
    is_start: bool
    is_end: bool
    connections: List["Node"]

    @staticmethod
    def create(name: str):
        return Node(name, name[0].isupper(), name == "start", name == "end", [])

    def connect(self, node: "Node"):
        if node not in self.connections:
            # print(f"{self.name} -> {node.name}")
            self.connections.append(node)
            node.connect(self)

    def __str__(self):
        return f"Node(name={self.name}, is_big={self.is_big}, is_start={self.is_start}, is_end={self.is_end}, connections={[n.name for n in self.connections]})"

@dataclass
class Path:
    sequence: List[Node]

    def can_add(self, node: Node):
        if node.is_big:
            return True
        else:
            return node not in self.sequence

    def __add__(self, node: Node):
        return Path(self.sequence[:] + [node])

    def __str__(self):
        return f"{[n.name for n in self.sequence]}"

@dataclass
class MoreComplexPath(Path):
    has_visited_small_cave_twice: bool

    def can_add(self, node: Node):
        if node.is_start:
            return False
        if node.is_big:
            return True
        else:
            if node not in self.sequence:
                return True
            else:
                if not self.has_visited_small_cave_twice:
                    return True
                else:
                    return False

    def __add__(self, node: Node):
        in_before = node in self.sequence
        if not self.has_visited_small_cave_twice and in_before and not node.is_big:
            return MoreComplexPath(self.sequence[:] + [node], True)
        return MoreComplexPath(self.sequence[:] + [node], self.has_visited_small_cave_twice)

class Graph:
    def __init__(self, lines):
        self.nodes = []
        for first_name, second_name in lines:
            first = self.find_or_create_node(first_name)
            second = self.find_or_create_node(second_name)
            first.connect(second)

    def find_or_create_node(self, name):
        node = self.find_node(name)
        if node is None:
            node = Node.create(name)
            self.nodes.append(node)
        return node

    def find_node(self, name) -> Optional[Node]:
        result = [n for n in self.nodes if n.name == name]
        if len(result) == 0:
            return None
        elif len(result) == 1:
            return result[0]
        else:
            raise ValueError(f"More than one node named {name}: {result}")

    def start(self) -> Node:
        result = self.find_node("start")
        if result is None:
            raise ValueError("No start node")
        return result

In [4]:
def walk(path: Path, node: Node):
    # print(f"Walking {path} {node}")
    if node.is_end:
        # print(f"Found route: {path}")
        return [path]

    result = []
    for c in node.connections:
        if path.can_add(c):
            result.extend(walk(path + c, c))
    return result

def find_paths(graph: Graph):
    result = []
    start = graph.start()
    path = Path([start])
    for c in start.connections:
        result.extend(walk(path + c, c))
    return result

def find_complex_paths(graph: Graph):
    result = []
    start = graph.start()
    path = MoreComplexPath([start], False)
    for c in start.connections:
        result.extend(walk(path + c, c))
    return result

In [5]:
paths = find_paths(Graph(parse_text(SAMPLE_TEXT_1)))
for p in paths:
    print(p)
len(paths)

['start', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'end']
['start', 'A', 'b', 'A', 'end']
['start', 'A', 'b', 'end']
['start', 'A', 'end']
['start', 'b', 'A', 'c', 'A', 'end']
['start', 'b', 'A', 'end']
['start', 'b', 'end']


10

In [6]:
len(find_paths(Graph(parse_text(SAMPLE_TEXT_2))))

19

In [7]:
len(find_paths(Graph(parse_text(SAMPLE_TEXT_3))))

226

In [8]:
len(find_paths(Graph(parse_text(read_input()))))

5457

In [9]:
len(find_complex_paths(Graph(parse_text(SAMPLE_TEXT_1))))

36

In [10]:
len(find_complex_paths(Graph(parse_text(SAMPLE_TEXT_2))))

103

In [11]:
len(find_complex_paths(Graph(parse_text(SAMPLE_TEXT_3))))

3509

In [12]:
len(find_complex_paths(Graph(parse_text(read_input()))))

128506